In [12]:
#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

>>> Downloading ollama...
######################################################################## 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#Download model
!ollama pull llama3

In [21]:
from tqdm import tqdm
import ollama
import pandas as pd
from datetime import date
import subprocess

In [ ]:
prompt = (
    "Hello, please act as a news analyst. I want to classify the article into one of 3 categories: increase-support/decrease-support/not-known. Based on article content tell me, if the stakeholders (e.g. world leaders) are extending their support to Ukraine (category increase-support) or diminishing it (category decrease-support). If this information is not clear from the article (e.g. the article reports on war development, but does not mention the reaction of other world leaders) respond with the category not-known. Your response must be only the category and nothing else. The article content: "
)

modfile = (
    'FROM llama3\n'
    f'SYSTEM "{prompt}"'
)

with open("/kaggle/working/Modfile", "w") as f:
    f.write(modfile)

subprocess.call("ollama create analyzer -f /kaggle/working/Modfile", shell=True)

In [113]:
try:
    # Attempt to kill process if ollama is already running
    process.kill()
except NameError:
    pass

process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.DEVNULL) #runs on a different thread

2024/06/05 14:00:04 routes.go:1007: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST: OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS: OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:*] OLLAMA_RUNNERS_DIR: OLLAMA_TMPDIR:]"
time=2024-06-05T14:00:04.577Z level=INFO source=images.go:729 msg="total blobs: 12"
time=2024-06-05T14:00:04.577Z level=INFO source=images.go:736 msg="total unused blobs removed: 0"
time=2024-06-05T14:00:04.577Z level=INFO source=routes.go:1053 msg="Listening on 127.0.0.1:11434 (version 0.1.41)"
time=2024-06-05T14:00:04.578Z level=INFO source=payload.go:30 msg="extracting embedded files" dir=/tmp/ollama3818707163/runn

In [115]:
def parse_war_news(start_date: date, end_date: date, source_whitelist, path='datasets/war_news.csv', ):
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'])
    df = df[
        (df['date'] >= start_date)
        & (df['date'] <= end_date)
        ]

    df = df[df['group_name'].isin(source_whitelist)]
    return df


def analyze_sentiment(text):
    response = ollama.generate(model='analyzer', prompt=f"{text}")
    return response['response']


def add_sentiment(df):
    df['sentiment'] = df['text'].apply(analyze_sentiment)
    return df

In [119]:
start_date = "2020-01-01"
end_date = "2023-12-31"

# Sources from https://www.kaggle.com/datasets/falloutbabe/russian-invasion-of-ukraine-live-news-dataset
path = '/kaggle/input/russian-invasion-of-ukraine-live-news-dataset/news.csv'

source_whitelist = [
    'Washington Post',
    'The New York Times'
]

war_news = parse_war_news(start_date, end_date, source_whitelist, path)
# war_news = war_news.sample(n=50)
print("Parsed war news: columns", war_news.columns, "shape", war_news.shape)

Parsed war news: columns Index(['id', 'group_id', 'group_name', 'text', 'date'], dtype='object') shape (4079, 5)


In [120]:
war_news = add_sentiment(war_news)
war_news.head()

,id,group_id,group_name,text,date,sentiment
294664,7,nytimes,The New York Times,Welcome to the Telegram channel from The New Y...,2022-03-11,not-known
294665,8,nytimes,The New York Times,The low rumble of heavy artillery fire echoed ...,2022-03-11,increase-support
294666,10,nytimes,The New York Times,"After a night of shelling, Ukrainians assess t...",2022-03-12,not-known
294667,11,nytimes,The New York Times,U.S. Officials Say Superyacht Could be Putin’s...,2022-03-12,increase-support
294668,12,nytimes,The New York Times,"Street battles hit a Kyiv suburb, some of the ...",2022-03-12,not-known


In [121]:
war_news.to_csv('/kaggle/working/war_news_sentiment_3.csv', index=False)
war_news

,id,group_id,group_name,text,date,sentiment
294664,7,nytimes,The New York Times,Welcome to the Telegram channel from The New Y...,2022-03-11,not-known
294665,8,nytimes,The New York Times,The low rumble of heavy artillery fire echoed ...,2022-03-11,increase-support
294666,10,nytimes,The New York Times,"After a night of shelling, Ukrainians assess t...",2022-03-12,not-known
294667,11,nytimes,The New York Times,U.S. Officials Say Superyacht Could be Putin’s...,2022-03-12,increase-support
294668,12,nytimes,The New York Times,"Street battles hit a Kyiv suburb, some of the ...",2022-03-12,not-known
...,...,...,...,...,...,...
386838,2568,nytimes,The New York Times,Here are some of the stories we're covering fr...,2023-07-03,not-known
386839,2881,washingtonpost,Washington Post,"Here's the latest from Ukraine:.- About 700,00...",2023-07-03,not-known
386840,2882,washingtonpost,Washington Post,"After mutiny, Kremlin looks to unwind holdings...",2023-07-03,not-known
386841,2883,washingtonpost,Washington Post,Celebrated Ukrainian war writer killed in Russ...,2023-07-03,not-known


In [122]:
# increase-support/decrease-support/not-known
pro_rus = war_news[war_news['sentiment'].str.contains('decrease-support')]
pro_rus

,id,group_id,group_name,text,date,sentiment
294672,17,nytimes,The New York Times,Ukraine Live Updates: Russian Airstrikes Kill ...,2022-03-13,decrease-support
294673,18,nytimes,The New York Times,"Brent Renaud, an American journalist, is kille...",2022-03-13,decrease-support
294674,19,nytimes,The New York Times,"Fearing beatings and arrests, Russians protest...",2022-03-13,decrease-support
294675,20,nytimes,The New York Times,Russia Asked China for Military and Economic A...,2022-03-13,decrease-support
294683,29,nytimes,The New York Times,Videos show civilian casualties in missile str...,2022-03-14,decrease-support
...,...,...,...,...,...,...
384218,2859,washingtonpost,Washington Post,A pro-Russian tide in Slovakia could threaten ...,2023-06-28,decrease-support
384852,2863,washingtonpost,Washington Post,"After Putin speech on deal with mercenaries, R...",2023-06-29,decrease-support
384853,2864,washingtonpost,Washington Post,"After Wagner mutiny, Navalny asks why he, not ...",2023-06-29,decrease-support
385506,2867,washingtonpost,Washington Post,Putin’s standing as global strongman in jeopar...,2023-06-30,decrease-support


In [127]:
pro_rus.iloc[1]['text']

'Brent Renaud, an American journalist, is killed in Ukraine.An award-winning filmmaker and journalist, Mr. Renaud, 50, was killed while reporting in a suburb of Kyiv. Another journalist was wounded.'